## Notebook to format wifi data files to make folium input easier

In [1]:
import pandas as pd

### Get building info

In [5]:
buildings_df = pd.read_csv('/home/calvinhathcock/Documents/College/UNCC/Fall 2022/ITSC 4155/Project/transformed_data/buildings.csv')

In [6]:
buildings_df

,Building abbreviation,Building Name,Id,Latitude,Longitude
0,Atki,Atkins Library,1,35.305893,-80.732145
1,Auxi,Auxiliary Services Building,2,35.307772,-80.730500
2,BaTF,NaN,3,NaN,NaN
3,Band,Barnhardt SAC,4,35.306302,-80.734409
4,BelG,Belk Gymnasium,5,35.305401,-80.735525
...,...,...,...,...,...
65,Wach,NaN,66,NaN,NaN
66,Wall,Wallis Hall,67,35.311493,-80.733694
67,Winn,Winningham,68,35.305137,-80.730419
68,With,Witherspoon Hall,69,35.310855,-80.732324


### Concatenate Raw building counts

In [7]:
counts_df = pd.read_csv('/home/calvinhathcock/Documents/College/UNCC/Fall 2022/ITSC 4155/Project/transformed_data/csv/counts_wireless_01-01-2021.csv')

In [8]:
counts_df

,timestamp,count,building
0,2021-01-01 00:00:00,8,Atki
1,2021-01-01 01:00:00,14,Atki
2,2021-01-01 02:00:00,2,Atki
3,2021-01-01 03:00:00,4,Atki
4,2021-01-01 04:00:00,2,Atki
...,...,...,...
1646,2021-01-01 19:00:00,66,Wood
1647,2021-01-01 20:00:00,38,Wood
1648,2021-01-01 21:00:00,50,Wood
1649,2021-01-01 22:00:00,60,Wood


In [9]:
counts_df.isna().any()

timestamp    False
count        False
building     False
dtype: bool

In [10]:
buildings_df

,Building abbreviation,Building Name,Id,Latitude,Longitude
0,Atki,Atkins Library,1,35.305893,-80.732145
1,Auxi,Auxiliary Services Building,2,35.307772,-80.730500
2,BaTF,NaN,3,NaN,NaN
3,Band,Barnhardt SAC,4,35.306302,-80.734409
4,BelG,Belk Gymnasium,5,35.305401,-80.735525
...,...,...,...,...,...
65,Wach,NaN,66,NaN,NaN
66,Wall,Wallis Hall,67,35.311493,-80.733694
67,Winn,Winningham,68,35.305137,-80.730419
68,With,Witherspoon Hall,69,35.310855,-80.732324


In [15]:
counts_df['Latitude'] = counts_df['building'].apply(lambda x: buildings_df[buildings_df['Building abbreviation'] == x].Latitude[0])

KeyError: 0

In [14]:
buildings_df[buildings_df['Building abbreviation'] == 'Atki'].Latitude[0]

35.30589304115362

## Convert pivoted buildings to desired format